In [1]:
import numpy as np
import pandas as pd
from scipy.sparse import coo_matrix
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

# Loading data

In [2]:
data_700_email = []
data_700_email = pd.read_csv("https://raw.githubusercontent.com/DangCao1999/ML-Study/master/BaiHoc/Naive%20Bayes/Spam%20email%20data/train-features-700.txt",  sep=" ", header=None)
data_700_email.head()

,0,1,2
0,1,19,2
1,1,45,1
2,1,50,1
3,1,75,1
4,1,85,1


In [3]:
label_700_email = []
label_700_email = pd.read_csv("https://raw.githubusercontent.com/DangCao1999/ML-Study/master/BaiHoc/Naive%20Bayes/Spam%20email%20data/train-labels-700.txt",  sep=" ", header=None)
label_700_email.head()

,0
0,0
1,0
2,0
3,0
4,0


# Tiền xử lý data

## Find email with number nword

In [4]:
def findEmailWithNumberNword(numberNWord):
    data_700_email_np = data_700_email.values
    data100Nword = []
    data_filter = {}
    for word in data_700_email_np:
        if(data_filter.get((word[0])) == None):
            data_filter[word[0]] = [word[1]]
        else:
            data_filter[word[0]].append(word[1])

        
    for keyEmail in data_filter:
        find = True
        for value in data_filter[keyEmail]:
            if(value > numberNWord):
                find = False
                break
        if(find):
            data100Nword.append(keyEmail)
        
    print(data100Nword)
    return data100Nword

**Tìm nword khác do không tìm được email nào với nword = 100 theo đề bài**

In [5]:
findEmailWithNumberNword(100)
findEmailWithNumberNword(500)
numberEmailFilter = findEmailWithNumberNword(1000)

[]
[]
[73, 152, 169, 176, 255, 383, 415, 489, 524, 591, 625]


**=> nword = 1000 sẽ có đc 11 email**

In [64]:
emailFiltered = pd.DataFrame();

**Lọc label tương ứng với email**

In [66]:
labelFilteredTemp = []
i = 0
for emailNumber in numberEmailFilter:
    #data
    dataFrame = data_700_email[data_700_email[0] == emailNumber]
    dataFrame.loc[:,0] = i
    emailFiltered = emailFiltered.append(dataFrame)
    i += 1
    #label
    labelFilteredTemp.append(label_700_email.loc[emailNumber].values)

In [67]:
print(emailFiltered)
labelFilterd = np.array(labelFilteredTemp)
labelFilterd = labelFilterd.reshape(11,)
print(labelFilterd)

        0    1  2
6037    0    4  1
6038    0   29  1
6039    0   35  1
6040    0   45  3
6041    0   56  1
...    ..  ... ..
72412  10  285  1
72413  10  529  1
72414  10  569  1
72415  10  705  1
72416  10  957  1

[266 rows x 3 columns]
[0 0 0 0 0 1 1 1 1 1 1]


# Sparse Matrix

In [69]:
nwords = 1000
nemails = 11
rows = emailFiltered[0] 
cols = emailFiltered[1]
data = emailFiltered[2]
coo_data = coo_matrix((data, (rows, cols)), shape=(nemails, nwords)).toarray()

In [70]:
print(coo_data)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


# Train - Test

In [74]:
train_feature, test_feature, train_label, test_label = train_test_split(coo_data, labelFilterd, test_size = 0.3, random_state=1)

In [75]:
MNB = MultinomialNB()
MNB.fit(train_feature, train_label)

MultinomialNB()

# 4. Predict - Evulation Accuracy

In [76]:
predict_label = MNB.predict(test_feature)
print(predict_label)

[0 1 0 1]


In [77]:
print('Training size = %d, accuracy = %.2f%%' % \
      (coo_data.shape[0], accuracy_score(test_label, predict_label)*100))

Training size = 11, accuracy = 75.00%
